<a href="https://colab.research.google.com/github/BryceRodgers7/CoTorch/blob/main/DuneBookTopics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install BERTopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 19.3 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 931.9 kB/s eta 0:00:00
     ━━━━━━

In [4]:
from bertopic import BERTopic
import json
import pandas as pd
import requests

url = 'https://raw.githubusercontent.com/ganesh-k13/shell/master/test_search/www.glozman.com/TextPages/Frank%20Herbert%20-%20Dune.txt'
response = requests.get(url)
lines = []
if response.status_code == 200:
    # Split the text into lines and remove any leading or trailing whitespace
    lines = response.text.splitlines()
    print(lines)
else:
    print('Failed to download file')

print(len(lines))
print(lines[15])
print(lines[1590])

topic_model = BERTopic(embedding_model='all-MiniLM-L6-v2')
topics, probs = topic_model.fit_transform(lines)




['Dune', 'Frank Herbert', '', 'Copyright 1965', '', 'Book 1', 'DUNE', '', '= = = = = = ', '', "A beginning is the time for taking the most delicate care that the balances are correct. This every sister of the Bene Gesserit knows. To begin your study of the life of Muad'Dib, then, take care that you first place him in his time: born in the 57th year of the Padishah Emperor, Shaddam IV. And take the most special care that you locate Muad'Dib in his place: the planet Arrakis. Do not be deceived by the fact that he was born on Caladan and lived his first fifteen years there. Arrakis, the planet known as Dune, is forever his place.", '-from "Manual of Muad\'Dib" by the Princess Irulan', '', '    In the week before their departure to Arrakis, when all the final scurrying about had reached a nearly unbearable frenzy, an old crone came to visit the mother of the boy, Paul.', '    It was a warm night at Castle Caladan, and the ancient pile of stone that had served the Atreides family as home fo

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
topic_model.get_topic_info()



,Topic,Count,Name,Representation,Representative_Docs
0,-1,2843,-1_the_and_of_to,"[the, and, of, to, he, in, it, that, his, paul]",[ Paul pressed past the man toward the dept...
1,0,648,0_jessica_her_she_herself,"[jessica, her, she, herself, thought, in, the,...","[ ""The Lady Jessica herself,"" the Baron sai..."
2,1,264,1_paul_mother_his_he,"[paul, mother, his, he, had, father, felt, him...",[ Paul looked down at the hand that had kno...
3,2,226,2_cool_him__,"[cool, him, , , , , , , , ]","[, , Him.]"
4,3,183,3_leto_duke_atreides_had,"[leto, duke, atreides, had, the, of, my, he, h...","[ ""Gurney,"" Leto said., The voice was D..."
...,...,...,...,...,...
125,124,11,124_gladiator_rautha_slave_feyd,"[gladiator, rautha, slave, feyd, feints, arena...","[ On his seventeenth birthday, Feyd-Rautha ..."
126,125,11,125_escort_spotters_duke_four,"[escort, spotters, duke, four, two, punched, c...","[ ""Spotters customarily keep their eyes on ..."
127,126,11,126_dig_sand_movement_ridge,"[dig, sand, movement, ridge, sandblasted, tons...",[ He saw her in the grayness matching him m...
128,127,10,127_sensible_whim_significance_squeeze,"[sensible, whim, significance, squeeze, long, ...","[ ""As long as you squeeze."" , ""You thin..."


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Topic'].plot(kind='hist', bins=20, title='Topic')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Count'].plot(kind='hist', bins=20, title='Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='index', y='Topic', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Topic']
  ys = series['Count']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('Topic', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Topic')
_ = plt.ylabel('Count')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Topic']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Topic'}, axis=1)
              .sort_values('Topic', ascending=True))
  xs = counted['Topic']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_8.sort_values('Topic', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Topic')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_9['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['Topic'].plot(kind='line', figsize=(8, 4), title='Topic')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['Count'].plot(kind='line', figsize=(8, 4), title='Count')
plt.gca().spines[['top', 'right']].set_visible(False)

In [7]:
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.7 MB/s eta 0:00:00


In [8]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [6]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

nltk.download('punkt')
nltk.download('stopwords')
print(string.punctuation)

# Initialize the Porter stemmer and get the list of English stop words
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def remove_punctuations(document):
  punctuations = string.punctuation
  return document.translate(str.maketrans('', '', punctuations))

def remove_stopwords(document):
  return " ".join([word for word in document.split() if word not in stop_words])

def remove_special_characters(document):
  document = re.sub('[^a-zA-Z0-9]', ' ', document)
  # limit spaces to 1
  document = re.sub('\s+', ' ', document)
  return document

def stem_words(document):
  return " ".join([stemmer.stem(word) for word in document.split()])

def preprocess_document(document):
    document = document.lower()
    document = remove_punctuations(document)
    document = remove_stopwords(document)
    document = remove_special_characters(document)
    processed_document = stem_words(document)
    # add in lemmatizing later?
    return processed_document

def preprocess_corpus(corpus):
    preprocessed_corpus = []
    for document in corpus:
        preprocessed_document = preprocess_document(document)
        preprocessed_corpus.append(preprocessed_document)
    return preprocessed_corpus


# Preprocess the corpus
preprocessed_corpus = preprocess_corpus(lines)

# Print the preprocessed corpus
for i, document in enumerate(preprocessed_corpus):
    print(f"Document {i+1}: {document}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Streaming output truncated to the last 5000 lines.
Document 3609: know signific
Document 3610: jessica turn sharpli stare son
Document 3611: father lie dead ruin arrakeen kyne said technic duke
Document 3612: im soldier imperium paul said technic hatchetman
Document 3613: kyne face darken even emperor sardaukar stand father bodi
Document 3614: sardaukar one thing legal sourc author anoth paul said
Document 3615: arraki way determin wear mantl author kyne said
Document 3616: jessica turn back look thought there steel man one taken temper weve need steel paul danger thing
Document 3617: paul said sardaukar arraki measur much belov emperor fear father give padishah emperor reason fear
Document 3618: lad kyne said thing dont
Document 3619: address sire lord paul said
Document 3620: gentli jessica thought
Document 3621: kyne stare paul jessica note glint admir planetologist face touch humor
Document 3622: sire kyne said
Document 3623: embarrass emperor paul said embarrass would divid arraki

In [19]:
def preprocess_data(documents):
  # Tokenize and remove stopwords
  texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in documents]
  return texts

werds = preprocess_data(preprocessed_corpus)


# be sure to split sentence before feed into Dictionary
# werds = [line.split() for line in werds]

# create dictionary
dictionary = corpora.Dictionary(werds)

# create 2nd copy of preprocessed??
texts = werds

# TDF
corpus = [dictionary.doc2bow(text) for text in texts]

# Set number of topics
num_topics = 3
# Build LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=10, alpha='auto', per_word_topics=True)


# Print the keywords for each topic
pprint(lda_model.print_topics())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.029*"paul" + 0.013*"jessica" + 0.010*"said" + 0.009*"thought" + '
  '0.007*"water" + 0.007*"man" + 0.007*"hand" + 0.007*"fremen" + 0.007*"look" '
  '+ 0.006*"could"'),
 (1,
  '0.027*"said" + 0.009*"one" + 0.008*"fremen" + 0.008*"see" + 0.008*"arraki" '
  '+ 0.007*"baron" + 0.007*"know" + 0.007*"harkonnen" + 0.007*"ask" + '
  '0.007*"bene"'),
 (2,
  '0.012*"sand" + 0.008*"shield" + 0.007*"rock" + 0.006*"dune" + 0.005*"maker" '
  '+ 0.005*"wall" + 0.004*"across" + 0.004*"surfac" + 0.004*"see" + '
  '0.004*"dust"')]


In [20]:

coherence_model_lda = CoherenceModel(model=lda_model, texts=werds, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence Score:  0.4950848461987687


In [ ]:
pyLDAvis.enable_notebook()
vis_data = gensimvis.prepare(lda_model, corpus, werds)

# Display the visualization
pyLDAvis.display(vis_data)